In [1]:
import pandas as pd 
import numpy as np

In [30]:
df=pd.read_csv("Greedy.csv")

In [31]:
df

,Task,Time Taken(Mins),Labour Requirement,Slots (Time Taken/15),Time Constraints (Start),Time Constraints (End)
0,Cleaning Bays,30,2,2,NaN,10:00
1,Stock Arranging,15,1,1,NaN,NaN
2,Stock Replenishment,15,1,1,NaN,NaN
3,Checklist Preparation,15,1,1,NaN,NaN
4,Sweeping Floor,30,2,2,9:00,9:30
5,Closing Readiness,30,2,2,10:30,NaN


In [19]:
#mapping time periods to slot numbers

Time_slots=pd.read_csv("Time_Slots.csv")

In [20]:
Time_slots

,Time_period,Time_Slot_start,Time_Slot_End
0,9:00,1.0,NaN
1,9:15,2.0,1.0
2,9:30,3.0,2.0
3,9:45,4.0,3.0
4,10:00,5.0,4.0
5,10:15,6.0,5.0
6,10:30,7.0,6.0
7,10:45,8.0,7.0
8,11:00,NaN,8.0


In [28]:
#Mapping start timing to time slots
mapping_start=dict(zip(Time_slots['Time_period'], Time_slots['Time_Slot_start']))

In [29]:
mapping_start


{'9:00': 1.0,
 '9:15': 2.0,
 '9:30': 3.0,
 '9:45': 4.0,
 '10:00': 5.0,
 '10:15': 6.0,
 '10:30': 7.0,
 '10:45': 8.0,
 '11:00': nan}

In [32]:
#Replacing time with mapped time slots
df_2=df.replace({"Time Constraints (Start)": mapping_start})

In [33]:
df_2

,Task,Time Taken(Mins),Labour Requirement,Slots (Time Taken/15),Time Constraints (Start),Time Constraints (End)
0,Cleaning Bays,30,2,2,NaN,10:00
1,Stock Arranging,15,1,1,NaN,NaN
2,Stock Replenishment,15,1,1,NaN,NaN
3,Checklist Preparation,15,1,1,NaN,NaN
4,Sweeping Floor,30,2,2,1.0,9:30
5,Closing Readiness,30,2,2,7.0,NaN


In [34]:
#Mapping end timings
mapping_end=dict(zip(Time_slots['Time_period'], Time_slots['Time_Slot_End']))

In [36]:
#Replacing time with mapped time slots

df_3=df_2.replace({"Time Constraints (End)": mapping_end})

In [37]:
df_3

,Task,Time Taken(Mins),Labour Requirement,Slots (Time Taken/15),Time Constraints (Start),Time Constraints (End)
0,Cleaning Bays,30,2,2,NaN,4.0
1,Stock Arranging,15,1,1,NaN,NaN
2,Stock Replenishment,15,1,1,NaN,NaN
3,Checklist Preparation,15,1,1,NaN,NaN
4,Sweeping Floor,30,2,2,1.0,2.0
5,Closing Readiness,30,2,2,7.0,NaN


In [39]:
#If there is no constraint on start time, then min timeslot by deadault is 1
df_3['Time Constraints (Start)']=df_3['Time Constraints (Start)'].fillna(1)

In [41]:
#If there is no constraint on end time, then min timeslot by deadault is 1

df_3['Time Constraints (End)']=df_3['Time Constraints (End)'].fillna(8)

In [42]:
df_3

,Task,Time Taken(Mins),Labour Requirement,Slots (Time Taken/15),Time Constraints (Start),Time Constraints (End)
0,Cleaning Bays,30,2,2,1.0,4.0
1,Stock Arranging,15,1,1,1.0,8.0
2,Stock Replenishment,15,1,1,1.0,8.0
3,Checklist Preparation,15,1,1,1.0,8.0
4,Sweeping Floor,30,2,2,1.0,2.0
5,Closing Readiness,30,2,2,7.0,8.0


In [54]:
#Getting Number of possible slots for each tasks
df_3['No_poss_Slots']=(df_3['Time Constraints (End)']-df_3["Time Constraints (Start)"]+1)-(df_3['Slots (Time Taken/15)']-1)

In [58]:
#sorting by no of possible slots for greedy to work optimally 
df_4=df_3.sort_values(by='No_poss_Slots').reset_index(drop=True)

In [94]:
#Repeating tasks based on number of slots 
df_5=df_4.loc[df_4.index.repeat(df_4['Slots (Time Taken/15)'])].reset_index(drop=True)

In [95]:
#Ranking them randomly
df_5['Rank']=df_5.groupby('Task')['Time Taken(Mins)'].rank('first').reset_index()['Time Taken(Mins)']

In [96]:
df_5

,Task,Time Taken(Mins),Labour Requirement,Slots (Time Taken/15),Time Constraints (Start),Time Constraints (End),No_poss_Slots,Rank
0,Sweeping Floor,30,2,2,1.0,2.0,1.0,1.0
1,Sweeping Floor,30,2,2,1.0,2.0,1.0,2.0
2,Closing Readiness,30,2,2,7.0,8.0,1.0,1.0
3,Closing Readiness,30,2,2,7.0,8.0,1.0,2.0
4,Cleaning Bays,30,2,2,1.0,4.0,3.0,1.0
5,Cleaning Bays,30,2,2,1.0,4.0,3.0,2.0
6,Stock Arranging,15,1,1,1.0,8.0,8.0,1.0
7,Stock Replenishment,15,1,1,1.0,8.0,8.0,1.0
8,Checklist Preparation,15,1,1,1.0,8.0,8.0,1.0


In [99]:
#Running the loop for greedy algorithm
for i in range(0,df_5.shape[0],1): 
            if df_5['Rank'][i]>1:
               Total=df_5.sum()
               Total.name='Total'
               df_5 = df_5.append(Total.transpose())
               df_5[['Time Constraints (Start)','Time Constraints (End)']]= df_5[['Time Constraints (Start)','Time Constraints (End)']].astype('int')
               cols=([*range(df_5['Time Constraints (Start)'][i],df_5['Time Constraints (End)'][i]+1,1)])
               df_5['Min'] = df_5[cols].idxmin(axis=1)
               df_5['Max']=df_5[cols].idxmax(axis=1)
               df_5[['Min','Max']]=df_5[['Min','Max']].astype('int')
               df_5['Max_row']=df_5['Max']+1
               a= df_5['Max_row'][i-1]
               df_5[a][i]=df_5["Labour Requirement"][i]
               df_5 = df_5.drop(labels='Total', axis=0)
#                print(i)
            else:    
                Total=df_5.sum()
                Total.name='Total'
                df_5 = df_5.append(Total.transpose())
                df_5[['Time Constraints (Start)','Time Constraints (End)']]= df_5[['Time Constraints (Start)','Time Constraints (End)']].astype('int')
                cols=([*range(df_5['Time Constraints (Start)'][i],df_5['Time Constraints (End)'][i]+1,1)])
                df_5['Min'] = df_5[cols].idxmin(axis=1)
                df_5['Max']=df_5[cols].idxmax(axis=1)
                df_5[['Min','Max']]=df_5[['Min','Max']].astype('int')
                df_5['Max_row']=df_5['Max']+1
                a=df_5['Min']['Total']
                df_5[a][i]=df_5["Labour Requirement"][i]
                df_5 = df_5.drop(labels='Total', axis=0)
#                 print(i)

    


<ipython-input-99-e66a7341aac2>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5[a][i]=df_5["Labour Requirement"][i]
<ipython-input-99-e66a7341aac2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5[a][i]=df_5["Labour Requirement"][i]


In [100]:
df_6=df_5[['Task',1,2,3,4,5,6,7,8]]

In [101]:
#The Final output
df_6

,Task,1,2,3,4,5,6,7,8
0,Sweeping Floor,2,0,0,0,0,0,0,0
1,Sweeping Floor,0,2,0,0,0,0,0,0
2,Closing Readiness,0,0,0,0,0,0,2,0
3,Closing Readiness,0,0,0,0,0,0,0,2
4,Cleaning Bays,0,0,2,0,0,0,0,0
5,Cleaning Bays,0,0,0,2,0,0,0,0
6,Stock Arranging,0,0,0,0,1,0,0,0
7,Stock Replenishment,0,0,0,0,0,1,0,0
8,Checklist Preparation,0,0,0,0,1,0,0,0
